<a href="https://colab.research.google.com/github/GustavoNicodemos/Algoritmo_analise_resultado/blob/main/An%C3%A1lise_GUV_V4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instale o pacote necessário
!pip install pyxlsb openpyxl

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest

In [ ]:
class AccountingAnalyzer:
    def __init__(self, historical_data):
        self.historical_data = historical_data.copy()

    def _consolidate_data(self, df):
        return df.groupby(['Cost_Center', 'Group_Account']).agg({'Amount_in_LC': 'sum'}).reset_index()

    def _detect_significant_variations(self, new_data):
        consolidated_new = self._consolidate_data(new_data)
        consolidated_hist = self._consolidate_data(self.historical_data)

        historical_std = self.historical_data.groupby(['Cost_Center', 'Group_Account'])['Amount_in_LC'].std().reset_index()
        historical_std.columns = ['Cost_Center', 'Group_Account', 'std_dev']

        comparativo = pd.merge(consolidated_new, consolidated_hist, on=['Cost_Center', 'Group_Account'], how='left', suffixes=('', '_hist'))
        comparativo = pd.merge(comparativo, historical_std, on=['Cost_Center', 'Group_Account'], how='left')

        comparativo['std_dev'] = comparativo['std_dev'].fillna(0)
        comparativo['Amount_in_LC_hist'] = comparativo['Amount_in_LC_hist'].fillna(0)

        comparativo['absolute_variation'] = comparativo['Amount_in_LC'] - comparativo['Amount_in_LC_hist']
        comparativo['percent_variation'] = np.where(
            comparativo['Amount_in_LC_hist'] != 0,
            (comparativo['absolute_variation'] / comparativo['Amount_in_LC_hist']) * 100,
            np.nan
        )

        comparativo['relevant'] = (comparativo['absolute_variation'] > comparativo['std_dev'] * 1.5) | \
                                  (comparativo['absolute_variation'] < -comparativo['std_dev'] * 1.5)

        detalhes_variacoes = comparativo[['Cost_Center', 'Group_Account', 'Amount_in_LC', 'Amount_in_LC_hist',
                                          'absolute_variation', 'percent_variation', 'std_dev', 'relevant']]

        variacoes_relevantes = comparativo[comparativo['relevant'] == True]

        return variacoes_relevantes, detalhes_variacoes

    def _detectar_lancamentos_explicativos(self, df_novo, df_historico, relevantes):
        explicativos = []

        for _, linha in relevantes.iterrows():
            cc = linha['Cost_Center']
            conta = linha['Group_Account']

            hist_lanc = df_historico[(df_historico['Cost_Center'] == cc) & (df_historico['Group_Account'] == conta)]
            novos_lanc = df_novo[(df_novo['Cost_Center'] == cc) & (df_novo['Group_Account'] == conta)]

            if hist_lanc.empty or novos_lanc.empty:
                continue

            media = hist_lanc['Amount_in_LC'].mean()
            std = hist_lanc['Amount_in_LC'].std()

            limiar = media + 1.5 * std

            for _, lanc in novos_lanc.iterrows():
                if lanc['Amount_in_LC'] > limiar or lanc['Amount_in_LC'] < -limiar:
                    explicativos.append(lanc)

        return pd.DataFrame(explicativos)

    def _generate_explanations(self, df):
        explanations = []
        for _, row in df.iterrows():
            if row.get('anomaly', 0) == -1:
                explanations.append(f"Anomalia isolada em {row['Group_Account']} no CC {row['Cost_Center']}")
            elif pd.notnull(row.get('percent_variation')) and abs(row['percent_variation']) > 10:
                explanations.append(f"Variação de {row['percent_variation']:.2f}% em {row['Group_Account']} no CC {row['Cost_Center']}")
            else:
                explanations.append("Sem variações significativas.")
        return explanations

    def analyze(self, new_data):
        consolidated_data = self._consolidate_data(new_data)

        modelo = IsolationForest(contamination=0.05, random_state=42)
        dados_para_modelo = pd.get_dummies(consolidated_data[['Cost_Center', 'Group_Account']])
        modelo.fit(dados_para_modelo)
        consolidated_data['anomaly'] = modelo.predict(dados_para_modelo)

        variacoes_relevantes, detalhes_variacoes = self._detect_significant_variations(new_data)
        explicativos_df = self._detectar_lancamentos_explicativos(new_data, self.historical_data, variacoes_relevantes)

        new_data['explanation'] = self._generate_explanations(new_data)

        return {
            'anomalias_df': explicativos_df,
            'consolidated_df': consolidated_data,
            'explicacoes_df': new_data[['Group_Account', 'explanation']],
            'variacoes_relevantes': variacoes_relevantes,
            'detalhes_variacoes': detalhes_variacoes
        }
    def analisar_variacoes_por_lancamento(self, df):
        df_proc = df.copy()
        df_proc['Mês'] = pd.to_datetime(df_proc['PERÍODO']).dt.to_period('M')

        # Cálculo da média e desvio padrão por conta
        stats = self.historical_data.groupby('Group_Account')['Amount_in_LC'].agg(['mean', 'std']).reset_index()
        stats = stats.rename(columns={'mean': 'media_historica', 'std': 'desvio_padrao'})

        # Agrega valor atual por conta
        valores_atuais = df_proc.groupby('Group_Account')['Amount_in_LC'].sum().reset_index()
        base_comparada = valores_atuais.merge(stats, on='Group_Account', how='left')
        base_comparada['var_relevante'] = abs(base_comparada['Amount_in_LC'] - base_comparada['media_historica']) > 2 * base_comparada['desvio_padrao']

        contas_relevantes = base_comparada[base_comparada['var_relevante']]['Group_Account'].tolist()

        # Dentro das contas com variação relevante, procura os lançamentos que mais contribuíram
        df_filtrado = df_proc[df_proc['Group_Account'].isin(contas_relevantes)].copy()
        df_filtrado['abs_valor'] = df_filtrado['Amount_in_LC'].abs()

        top_lancamentos = (
            df_filtrado
            .groupby(['Group_Account', 'Text', 'Cost_Center'])
            .agg(qtd=('Amount_in_LC', 'count'),
                 soma_valor=('Amount_in_LC', 'sum'),
                 soma_abs=('abs_valor', 'sum'))
            .reset_index()
            .sort_values(['Group_Account', 'soma_abs'], ascending=[True, False])
        )

        return top_lancamentos, base_comparada

In [ ]:
# Execução no Colab
if __name__ == "__main__":
    base_dados = pd.read_excel('Dados_GUV.xlsb')
    dados_historicos = pd.read_excel('Dados_historicos_GUV.xlsb')

    analisador = AccountingAnalyzer(historical_data=dados_historicos)

    relatorio = analisador.analyze(base_dados)

    # Novo método de análise de lançamentos responsáveis por variações
    lancamentos_relevantes, resumo_relevantes = analisador.analisar_variacoes_por_lancamento(base_dados)

    # Exportação para Excel com todas as abas
    with pd.ExcelWriter('Relatorio_Contabil_Analitico_Completo.xlsx') as writer:
        relatorio['anomalias_df'].to_excel(writer, sheet_name='Anomalias', index=False)
        relatorio['consolidated_df'].to_excel(writer, sheet_name='Consolidado_DRE', index=False)
        relatorio['explicacoes_df'].drop_duplicates().to_excel(writer, sheet_name='Explicacoes', index=False)
        relatorio['variacoes_relevantes'].to_excel(writer, sheet_name='Variacoes_Relevantes', index=False)
        relatorio['detalhes_variacoes'].to_excel(writer, sheet_name='Detalhamento_Lancamentos', index=False)
        lancamentos_relevantes.to_excel(writer, sheet_name='Lancamentos_Relevantes', index=False)
        resumo_relevantes.to_excel(writer, sheet_name='Resumo_Por_Centro_Conta', index=False)

    print("✅ Planilha final gerada com múltiplas abas, incluindo análise aprofundada de variações.")

✅ Planilha final gerada com múltiplas abas, incluindo análise aprofundada de variações.


In [ ]:
import os

# Verifica se o arquivo foi criado
print(os.listdir())
from google.colab import files
files.download('Relatorio_Contabil_Analitico_Completo.xlsx')


['.config', '.ipynb_checkpoints', 'Dados_GUV.xlsb', 'Relatorio_Contabil_Analitico_Completo.xlsx', 'Dados_historicos_GUV.xlsb', 'sample_data']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>